In [106]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움4에서 나타나는 오류 잡기위함
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

### DB연결용
import pymysql
import pandas as pd
from tqdm import tqdm

### web driver test

In [2]:
def setup_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
    chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

    # install Chrome webdriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

driver = setup_chrome_driver()

C:\Users\Chaaaaabbo\AppData\Local\Temp\ipykernel_29292\1533073334.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/Chaaaaabbo/Desktop/chromedriver_win32/chromedriver.exe")


### 보배드림

In [105]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

# Set up Chrome webdriver
driver = webdriver.Chrome(options=chrome_options)

#Selenium_driver로 url 접속
url = "https://www.bobaedream.co.kr/search"
driver.get(url)
#Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
time.sleep(1)

x_path_search = '/html/body/div/div[2]/span/form/input[6]'
searchbox = driver.find_element(By.XPATH, x_path_search)
searchbox.click()
element = driver.find_element(By.NAME, "keyword")
element.send_keys("맥북")
element.submit()

# 검색 정렬 조건 정확도로 바꿈
x_path_acc = '/html/body/div/div[3]/div[1]/div[1]/ul/li[1]/a' 
acc_box = driver.find_element(By.XPATH, x_path_acc)
acc_box.click()

#BeautifulSoup_html을 Parsing 함
soup = BeautifulSoup(driver.page_source, 'html.parser')

# title은 
search_in_commu = soup.find('div', "search_Community")
search_results = search_in_commu.findAll("a")

data = {}
cnt = 0
for idx, result in enumerate(search_results):
    if idx >= 3: #0:전체, 1:제목, 2:글쓴이
        data_ = {}
        data_['title'] = result.text
        data_['url'] = f"https://www.bobaedream.co.kr/{result['href']}"
        if data_['title'] != '':
            data[cnt] = data_
            cnt += 1
        if cnt == 3:
            break

search_results = search_in_commu.findAll("span","next")
cnt = 0
for idx, result in enumerate(search_results):
    if idx % 2 == 1:
        data[cnt]["date"] = result.text
        cnt += 1
        if cnt == 3:
            break
print(data)

{0: {'title': '맥북 사전예약중 참고하세요', 'url': 'https://www.bobaedream.co.kr//view?code=freeb&No=2454362', 'date': '21. 11. 15'}, 1: {'title': '맥북인데 윈도깔아쓰는 제 맥북..ㅠ ㅠ', 'url': 'https://www.bobaedream.co.kr//view?code=national&No=334380', 'date': '11. 12. 12'}, 2: {'title': '애플맥북', 'url': 'https://www.bobaedream.co.kr//view?code=freeb&No=2451286', 'date': '21. 11. 11'}}


### 인스티즈
---

In [113]:
def instiz_crawling(word_list):
    for word in word_list:
        url = f"https://www.instiz.net/popup_search.htm#gsc.tab=0&gsc.q={word}&gsc.sort="
        driver.get(url)

        # Parse the HTML response using Beautiful Soup to extract the data of each result
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all("a", class_="gs-title")

        data = {}
        temp = None
        cnt = 0
        try: 
            for result in results:
                data_ = {}
                title = result.text.split('-')[0][:-1]
                if title == temp: # 이전 내용과 동일하면 pass
                    pass
                else:
                    if title == '':
                        break
                    data_['title'] = temp = title
                    data_['url'] = result['href']
                    data[cnt] = data_
                    cnt += 1
        except Exception as e:
            temp = None
            print('검색결과가 없습니다.', e)
        # Close the Chrome webdriver
        driver.quit()

    return data # 나중에 MySQL 어떻게 넣을지 고민하고 형태 바꿔야함

In [114]:
word_list = ['맥북']
instiz_crawling(word_list)

{0: {'title': '맥북에 청포도에이슬 쏟은 트위터리안 후기.JPG',
  'url': 'https://www.instiz.net/pt/7004209'},
 1: {'title': '미친듯이 갈리는 애플 맥북 색상', 'url': 'https://www.instiz.net/pt/7215722'},
 2: {'title': '업자한테 맥북 중고거래(판매)한 후기',
  'url': 'https://www.instiz.net/pt/6541040'},
 3: {'title': '맥북 꼭 사라 ..... 나도 노트북은 당연히 윈도우가 짱일줄',
  'url': 'https://www.instiz.net/name/43138973'},
 4: {'title': '와 류수정 인스타 맥북 감성 오진다',
  'url': 'https://www.instiz.net/name_enter/75500207'},
 5: {'title': '오빠한테 맥북사달랬더니 진짜사줌.Manhwa',
  'url': 'https://www.instiz.net/pt/7054551'},
 6: {'title': '우리집 고양이련이 맥북 떨궛다', 'url': 'https://www.instiz.net/pt/7064012'},
 7: {'title': '애플 가로수길 매장에서 맥북 박살낸 썰',
  'url': 'https://www.instiz.net/pt/6876481'},
 8: {'title': '판에서 핫한 맥북 보상사건...jpg',
  'url': 'https://www.instiz.net/pt/4010864'},
 9: {'title': '맥북 제스처 모음.gif', 'url': 'https://www.instiz.net/pt/4668416'}}